In [1]:
# 01. Setup 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Para Regressão Linear/Estatística
import statsmodels.api as sm 

# Para Modelos de ML e Regularização
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

In [2]:
# 02. Carregamento do mtcars (Para Regressão Linear e Polinomial)
mtcars = sm.datasets.get_rdataset("mtcars", "datasets").data
print("Shape do mtcars:", mtcars.shape)
print(mtcars.head())

# Para o Boston Housing (Usaremos o Scikit-learn para os modelos de regularização)
from sklearn.datasets import fetch_california_housing
# Observação: Para regularização, usaremos o dataset California Housing (alternativa moderna ao Boston Housing)
# que é similarmente complexo e widely-used no sklearn.
housing_data = fetch_california_housing(as_frame=True)
boston_df = housing_data.frame
boston_df['MEDV'] = boston_df['MedHouseVal'] # Nome da variável alvo similar ao Boston
print("\nShape do California Housing:", boston_df.shape)
print(boston_df.head())

Shape do mtcars: (32, 11)
                    mpg  cyl   disp   hp  drat     wt   qsec  vs  am  gear  \
rownames                                                                     
Mazda RX4          21.0    6  160.0  110  3.90  2.620  16.46   0   1     4   
Mazda RX4 Wag      21.0    6  160.0  110  3.90  2.875  17.02   0   1     4   
Datsun 710         22.8    4  108.0   93  3.85  2.320  18.61   1   1     4   
Hornet 4 Drive     21.4    6  258.0  110  3.08  3.215  19.44   1   0     3   
Hornet Sportabout  18.7    8  360.0  175  3.15  3.440  17.02   0   0     3   

                   carb  
rownames                 
Mazda RX4             4  
Mazda RX4 Wag         4  
Datsun 710            1  
Hornet 4 Drive        1  
Hornet Sportabout     2  

Shape do California Housing: (20640, 10)
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.

In [3]:
# 03. Definindo as variáveis preditoras (X) e alvo (y)
X_mtcars = mtcars[['wt', 'hp']]
y_mtcars = mtcars['mpg']

# Adicionando a constante (Intercepto) - exigência do statsmodels
X_mtcars_sm = sm.add_constant(X_mtcars)

# Treinando o modelo OLS (Ordinary Least Squares)
linear_model = sm.OLS(y_mtcars, X_mtcars_sm).fit()

print("--- Regressão Linear Múltipla (mpg ~ wt + hp) ---")
print(linear_model.summary().tables[1]) # Tabela de Coeficientes
print(f"\nR-quadrado Ajustado: {linear_model.rsquared_adj:.4f}")
print(f"RSE (Residual Standard Error, equivalente a MSE^0.5): {np.sqrt(linear_model.mse_resid):.3f}")

--- Regressão Linear Múltipla (mpg ~ wt + hp) ---
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         37.2273      1.599     23.285      0.000      33.957      40.497
wt            -3.8778      0.633     -6.129      0.000      -5.172      -2.584
hp            -0.0318      0.009     -3.519      0.001      -0.050      -0.013

R-quadrado Ajustado: 0.8148
RSE (Residual Standard Error, equivalente a MSE^0.5): 2.593


In [4]:
# 04. Exemplo de Previsão com Regressão Linear Múltipla 

# 1. Definir o novo dado (Peso = 3.000 lbs, Horsepower = 150)
novo_dado = pd.DataFrame({
    'wt': [3.00], 
    'hp': [150]
})

# 2. Adicionar a constante (Intercepto) - exigência do statsmodels
novo_dado_sm = sm.add_constant(novo_dado, has_constant='add')

# 3. Fazer a previsão
previsao_mpg = linear_model.predict(novo_dado_sm)

print(f"Previsão de MPG para o novo veículo (wt=3.0, hp=150): {previsao_mpg[0]:.2f}")

Previsão de MPG para o novo veículo (wt=3.0, hp=150): 20.83


In [5]:
# 05. Regressão Polinomial de 3º Grau (mpg ~ hp + hp^2)

# Variável única (hp)
X_poly = mtcars[['hp']]
y_poly = mtcars['mpg'] # Target (endog)

# 1. Criar as features polinomiais de 2º grau (hp e hp^2)
poly_transformer = PolynomialFeatures(degree=2, include_bias=False)
X_poly_2nd = poly_transformer.fit_transform(X_poly)
X_poly_2nd_df = pd.DataFrame(X_poly_2nd, columns=['hp', 'hp_squared'])

# 1.1.Atribuir o índice original (nomes dos carros)
X_poly_2nd_df.index = X_poly.index 

# 2. Adicionar a constante para statsmodels
X_poly_sm = sm.add_constant(X_poly_2nd_df) # Exog

# 3. Treinar o modelo Polinomial
poly_model = sm.OLS(y_poly, X_poly_sm).fit()

print("--- Regressão Polinomial (mpg ~ hp + hp^2) ---")
print(poly_model.summary().tables[1])
print(f"\nR-quadrado Ajustado: {poly_model.rsquared_adj:.4f}")
print(f"RSE: {np.sqrt(poly_model.mse_resid):.3f}")

--- Regressão Polinomial (mpg ~ hp + hp^2) ---
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         40.4091      2.741     14.744      0.000      34.804      46.015
hp            -0.2133      0.035     -6.115      0.000      -0.285      -0.142
hp_squared     0.0004   9.84e-05      4.275      0.000       0.000       0.001

R-quadrado Ajustado: 0.7393
RSE: 3.077


In [6]:
# 06. Regressão Polinomial de 3º Grau (mpg ~ hp + hp^2 + hp^3) 

# Variável única (hp)
X_poly = mtcars[['hp']]
y_poly = mtcars['mpg']

# 1. Criar as features polinomiais de 3º grau
poly_transformer_3rd = PolynomialFeatures(degree=3, include_bias=False)
X_poly_3rd = poly_transformer_3rd.fit_transform(X_poly)
X_poly_3rd_df = pd.DataFrame(X_poly_3rd)

# 1.1. Atribuir o índice original para evitar o erro de desalinhamento (como corrigimos antes)
X_poly_3rd_df.index = X_poly.index 
X_poly_3rd_df.columns = ['hp', 'hp_squared', 'hp_cubed']

# 2. Adicionar a constante para statsmodels
X_poly_3rd_sm = sm.add_constant(X_poly_3rd_df)

# 3. Treinar o modelo Polinomial de 3º grau
poly_model_3rd = sm.OLS(y_poly, X_poly_3rd_sm).fit()

print("--- Regressão Polinomial de 3º Grau (mpg ~ hp + hp^2 + hp^3) ---")
print(poly_model_3rd.summary().tables[1]) # Tabela de Coeficientes e P-valores
print(f"\nR-quadrado Ajustado: {poly_model_3rd.rsquared_adj:.4f}")
print(f"RSE: {np.sqrt(poly_model_3rd.mse_resid):.3f}")

--- Regressão Polinomial de 3º Grau (mpg ~ hp + hp^2 + hp^3) ---
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         44.2249      5.961      7.419      0.000      32.015      56.435
hp            -0.2945      0.118     -2.500      0.019      -0.536      -0.053
hp_squared     0.0009      0.001      1.328      0.195      -0.000       0.002
hp_cubed   -8.701e-07    1.2e-06     -0.722      0.476   -3.34e-06     1.6e-06

R-quadrado Ajustado: 0.7349
RSE: 3.103


In [7]:
# --- CÉLULA 7: CORREÇÃO DO PRÉ-PROCESSAMENTO (Anti-Data Leakage) ---

# Variáveis X e y
X_reg = boston_df.drop(['MEDV', 'MedHouseVal'], axis=1) 
y_reg = boston_df['MEDV'] # Mantém 'MEDV' como o alvo

# Padronização (Crucial para Regularização)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_reg)

# Split em Treino e Teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_reg, test_size=0.3, random_state=42
)
print("Dados prontos para Regularização: Treino (X, y) e Teste (X, y) criados.")

Dados prontos para Regularização: Treino (X, y) e Teste (X, y) criados.


In [8]:
# 08. Regressão Ridge(L2) com Cross Validation
from sklearn.linear_model import RidgeCV

# Grid de alphas (lambdas) a serem testados
alphas = np.logspace(-3, 3, 100) # 100 valores entre 0.001 e 1000

# RidgeCV usa Validação Cruzada (CV=5) para encontrar o alpha ideal
ridge_cv = RidgeCV(alphas=alphas, cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X_train, y_train)

# Melhor alpha (lambda) encontrado
print(f"Ridge: Melhor Alpha (lambda) encontrado: {ridge_cv.alpha_:.4f}")

# Avaliação
y_pred_ridge = ridge_cv.predict(X_test)
rmse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print(f"RMSE (Teste) - Ridge: {rmse_ridge:.4f}")
print(f"R2 (Teste) - Ridge: {r2_ridge:.4f}")

# Coeficientes
print("Coeficientes (Ridge):")
print(pd.Series(ridge_cv.coef_, index=X_reg.columns).sort_values(ascending=False))

Ridge: Melhor Alpha (lambda) encontrado: 4.3288
RMSE (Teste) - Ridge: 0.5305
R2 (Teste) - Ridge: 0.5959
Coeficientes (Ridge):
MedInc        0.846624
AveBedrms     0.367473
HouseAge      0.122234
Population   -0.000748
AveOccup     -0.035019
AveRooms     -0.300895
Longitude    -0.865170
Latitude     -0.890295
dtype: float64


In [9]:
# 09. Regressão Lasso (L1) 
from sklearn.linear_model import LassoCV

# Grid de alphas (lambdas) a serem testados (a mesma faixa usada para o Ridge)
alphas = np.logspace(-3, 3, 100) 

# LassoCV usa Validação Cruzada (CV=5) para encontrar o alpha ideal
# max_iter aumentado para garantir convergência em penalidades mais fracas
lasso_cv = LassoCV(alphas=alphas, cv=5, max_iter=10000, random_state=42)
lasso_cv.fit(X_train, y_train)

# Melhor alpha (lambda) encontrado
print(f"Lasso: Melhor Alpha (lambda) encontrado: {lasso_cv.alpha_:.4f}")

# Avaliação
y_pred_lasso = lasso_cv.predict(X_test)
# Usando np.sqrt para garantir o cálculo correto do RMSE
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso)) 
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f"RMSE (Teste) - Lasso: {rmse_lasso:.4f}")
print(f"R2 (Teste) - Lasso: {r2_lasso:.4f}")

# Coeficientes
print("Coeficientes (Lasso - Zera Features Irrelevantes):")
coefs_lasso = pd.Series(lasso_cv.coef_, index=X_reg.columns)
# Imprimir apenas os coeficientes zerados (seleção de features)
print(coefs_lasso[coefs_lasso == 0]) 
print("\nTodos os Coeficientes (para comparação com Ridge):")
print(coefs_lasso.sort_values(ascending=False))

Lasso: Melhor Alpha (lambda) encontrado: 0.0027
RMSE (Teste) - Lasso: 0.7270
R2 (Teste) - Lasso: 0.5973
Coeficientes (Lasso - Zera Features Irrelevantes):
Population   -0.0
dtype: float64

Todos os Coeficientes (para comparação com Ridge):
MedInc        0.832266
AveBedrms     0.328678
HouseAge      0.123207
Population   -0.000000
AveOccup     -0.033060
AveRooms     -0.264108
Longitude    -0.839062
Latitude     -0.866180
dtype: float64


In [10]:
# 10. Elastic Net (L1 + L2) 
from sklearn.linear_model import ElasticNetCV

# Grid de l1_ratio (mistura) a serem testados (0=Ridge, 1=Lasso)
l1_ratios = [0.01, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1.0]

# Grid de alphas (lambdas)
alphas = np.logspace(-3, 3, 100)

# ElasticNetCV usa Validação Cruzada para encontrar o melhor alpha E o melhor l1_ratio
enet_cv = ElasticNetCV(
    l1_ratio=l1_ratios, 
    alphas=alphas, 
    cv=5, 
    max_iter=10000, 
    random_state=42
)
enet_cv.fit(X_train, y_train)

# Melhores hiperparâmetros
print(f"Elastic Net: Melhor Alpha (lambda) encontrado: {enet_cv.alpha_:.4f}")
print(f"Elastic Net: Melhor L1_Ratio (alpha no R) encontrado: {enet_cv.l1_ratio_:.4f}")

# Avaliação
y_pred_enet = enet_cv.predict(X_test)
# Usando np.sqrt para garantir o cálculo correto do RMSE
rmse_enet = np.sqrt(mean_squared_error(y_test, y_pred_enet))
r2_enet = r2_score(y_test, y_pred_enet)

print(f"RMSE (Teste) - Elastic Net: {rmse_enet:.4f}")
print(f"R2 (Teste) - Elastic Net: {r2_enet:.4f}")

# Coeficientes
print("Coeficientes (Elastic Net):")
print(pd.Series(enet_cv.coef_, index=X_reg.columns).sort_values(ascending=False))

Elastic Net: Melhor Alpha (lambda) encontrado: 0.0027
Elastic Net: Melhor L1_Ratio (alpha no R) encontrado: 1.0000
RMSE (Teste) - Elastic Net: 0.7270
R2 (Teste) - Elastic Net: 0.5973
Coeficientes (Elastic Net):
MedInc        0.832266
AveBedrms     0.328678
HouseAge      0.123207
Population   -0.000000
AveOccup     -0.033060
AveRooms     -0.264108
Longitude    -0.839062
Latitude     -0.866180
dtype: float64
